### Importing the Necessary Libraries

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow
from tensorflow import keras
from keras import Input
from keras.applications import DenseNet121
from keras.models import Model
from keras.utils import image_dataset_from_directory, load_img
from keras.utils.image_utils import img_to_array
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import Accuracy, CategoricalAccuracy, AUC
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization

### Splitting the Dataset into Training Set and Test Set

In [2]:
trainingDataset = image_dataset_from_directory(
    '../Datasets/COVID-19_Chest_CT-Scan_Dataset/',
    labels = 'inferred',
    label_mode = 'categorical',
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (224, 224),
    seed = 123,
    subset = 'training',
    validation_split = 0.2
)

testDataset = image_dataset_from_directory(
    '../Datasets/COVID-19_Chest_CT-Scan_Dataset/',
    labels = 'inferred',
    label_mode = 'categorical',
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (224, 224),
    seed = 123,
    subset = 'validation',
    validation_split = 0.2
)

Found 2481 files belonging to 2 classes.
Using 1985 files for training.
Found 2481 files belonging to 2 classes.
Using 496 files for validation.


### Building the DenseNet121 Architecture Model

In [3]:
denseNet = DenseNet121(include_top = False, weights = 'imagenet', input_tensor = Input(shape = (224, 224, 3)), input_shape = (224, 224, 3), classifier_activation = 'softmax')

denseModel = denseNet.output
denseModel = GlobalAveragePooling2D()(denseModel)
denseModel = Flatten()(denseModel)
denseModel = Dense(units = 512, activation = 'relu')(denseModel)
denseModel = Dropout(rate = 0.25)(denseModel)
denseModel = Dense(units = 2, activation = 'softmax')(denseModel)

model = Model(inputs = denseNet.input, outputs = denseModel)

for layer in denseNet.layers :
    layer.trainable = False

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [4]:
model.compile(optimizer = Adam(learning_rate = 1e-3), loss = CategoricalCrossentropy(), metrics = [Accuracy(), CategoricalAccuracy(), AUC()])

In [7]:
model_checkpoint = ModelCheckpoint('./SavedModels/Pretrained_DenseNet121/DenseNet121_BestModel.h5', monitor = 'val_categorical_accuracy', verbose = 1, save_best_only = True, mode = 'auto')

finalModel = model.fit(x = trainingDataset, validation_data = testDataset, batch_size = 32, epochs = 5, verbose = 'auto', callbacks = [model_checkpoint])

keras.models.save_model(model, './SavedModels/Pretrained_DenseNet121/DenseNet121_BestModel.h5', overwrite = True, include_optimizer = True, save_format = None, signatures = None, options = None)

Epoch 1/5
63/63 [==============================] - ETA: 0s - loss: 0.2981 - accuracy: 0.0000e+00 - categorical_accuracy: 0.8730 - auc: 0.9456
Epoch 1: val_categorical_accuracy improved from -inf to 0.87702, saving model to ./SavedModels/Pretrained_DenseNet121\DenseNet121_BestModel.h5
63/63 [==============================] - 394s 6s/step - loss: 0.2981 - accuracy: 0.0000e+00 - categorical_accuracy: 0.8730 - auc: 0.9456 - val_loss: 0.2877 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.8770 - val_auc: 0.9500
Epoch 2/5
63/63 [==============================] - ETA: 0s - loss: 0.2650 - accuracy: 0.0000e+00 - categorical_accuracy: 0.8912 - auc: 0.9577
Epoch 2: val_categorical_accuracy did not improve from 0.87702
63/63 [==============================] - 404s 6s/step - loss: 0.2650 - accuracy: 0.0000e+00 - categorical_accuracy: 0.8912 - auc: 0.9577 - val_loss: 0.3071 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.8609 - val_auc: 0.9429
Epoch 3/5
63/63 [=====================

### Evaluating the Final Model

In [8]:
loss, accuracy, catAccuracy, auc = model.evaluate(testDataset, verbose = 'auto', batch_size = 32)
print("Loss: {:.2f} Accuracy: {:.2f} Categorical Accuracy: {:.2f} AUC: {:.2f}".format(loss, accuracy, catAccuracy, auc))

16/16 [==============================] - 73s 5s/step - loss: 0.2290 - accuracy: 0.0000e+00 - categorical_accuracy: 0.9032 - auc: 0.9692
Loss: 0.23 Accuracy: 0.00 Categorical Accuracy: 0.90 AUC: 0.97


### Making the Predictions

In [13]:
testImage = load_img(path = '../Datasets/COVID-19_Chest_CT-Scan_Dataset/COVID/Covid (24).png', target_size = (224, 224))
testImage = img_to_array(testImage)
testImage = np.array([testImage])
prediction = model.predict(testImage)
print(prediction)

1/1 [==============================] - 0s 248ms/step
[[0.89480716 0.10519277]]
